<a href="https://colab.research.google.com/github/hzang41981/ESC-50/blob/master/HW2P2/HW2P2_Convnext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}
%cd /content
!mkdir drive
%cd drive
!mkdir MyDrive
%cd ..
%cd ..
!google-drive-ocamlfuse /content/drive/MyDrive

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 155320 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.27-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.27-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
import torchvision.transforms as ttf

import os
import os.path as osp

from tqdm import tqdm
from PIL import Image
from sklearn.metrics import roc_auc_score
import numpy as np

# TODOs
As you go, please read the code and keep an eye out for TODOs!

# Download Data

In [ ]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:
    f.write('{"username":"hzang41981","key":"07ce2b8c32b2a27182316ae7fe638b80"}') # Put your kaggle username & key here

!chmod 600 /root/.kaggle/kaggle.json

     |████████████████████████████████| 59 kB 2.3 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.8-py3-none-any.whl size=73275 sha256=8736accb10ae99c39b8e10620ae0c4bc76e391e07ae3fdda1e8574683998a48c
  Stored in directory: /root/.cache/pip/wheels/de/f7/d8/c3902cacb7e62cb611b1ad343d7cc07f42f7eb76ae3a52f3d1
Successfully built kaggle
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [ ]:
!kaggle competitions download -c 11-785-s22-hw2p2-classification
!kaggle competitions download -c 11-785-s22-hw2p2-verification

!unzip -q 11-785-s22-hw2p2-classification.zip
!unzip -q 11-785-s22-hw2p2-verification.zip

!ls

100% 2.35G/2.35G [00:32<00:00, 122MB/s]
100% 2.35G/2.35G [00:32<00:00, 78.7MB/s]
 96% 252M/263M [00:01<00:00, 198MB/s]
100% 263M/263M [00:01<00:00, 176MB/s]
11-785-s22-hw2p2-classification.zip   sample_data
11-785-s22-hw2p2-verification.zip     train_subset
classification			      verification
classification_sample_submission.csv  verification_sample_submission.csv


# Hyperparameters

In [6]:
"""
The well-accepted SGD batch_size & lr combination for CNN classification is 256 batch size for 0.1 learning rate.
When changing batch size for SGD, follow the linear scaling rule - halving batch size -> halve learning rate, etc.
This is less theoretically supported for Adam, but in my experience, it's a decent ballpark estimate.
"""
batch_size = 256
lr = 0.1
epochs = 20 # Just for the early submission. We'd want you to train like 50 epochs for your main submissions.

In [21]:
from audioop import bias
from tokenize import group
import torch
import torch.nn as nn
import math


class InvertedResidualBlock(nn.Module):
    def __init__(self,
                 in_channels,
                 out_channels,
                 stride,
                 expand_ratio):
        super().__init__() # Just have to do this for all nn.Module classes

        # Can only do identity residual connection if input & output are the
        # same channel & spatial shape.
        if stride == 1 and in_channels == out_channels:
            self.do_identity = True
        else:
            self.do_identity = False
        
        # Expand Ratio is like 6, so hidden_dim >> in_channels
        hidden_dim = in_channels * expand_ratio

        self.spatial_mixing = nn.Sequential(
            # TODO: Fill this in!
            nn.Conv2d(in_channels,in_channels,kernel_size=7,padding=3,groups=in_channels,bias=False),
            nn.BatchNorm2d(in_channels)
        )

       
        self.feature_mixing = nn.Sequential(
            # TODO: Fill this in!
            nn.Conv2d(in_channels,hidden_dim,kernel_size=1,stride=1,padding=0,bias=False),
            nn.GELU()
        )

        self.bottleneck_channels = nn.Sequential(
            # TODO: Fill this in!
            nn.Conv2d(hidden_dim,out_channels,kernel_size=1,stride=1,padding=0,bias=False)
        )

    def forward(self, x):
        out = self.spatial_mixing(x)
        out = self.feature_mixing(out)
        out= self.bottleneck_channels(out)

        if self.do_identity:
            return x + out
        else:
            return out

class ConvNeXt(nn.Module):

    def __init__(self, num_classes= 7000):
        super().__init__()

        self.num_classes = num_classes

        """
        First couple of layers are special, just do them here.
        This is called the "stem". Usually, methods use it to downsample or twice.
        """

        self.stem = nn.ModuleList()
        stem = nn.Sequential(
            # TODO: Fill this in!
            nn.Conv2d(3,96,kernel_size=4,stride=4,bias=False),
            nn.BatchNorm2d(96)
        )
        self.stem.append(stem)

        #down_sample
        res1=nn.Sequential(
            nn.BatchNorm2d(96),
            nn.Conv2d(96,192,kernel_size=2,stride=2,bias=False),)
        self.stem.append(res1)
        
        res2=nn.Sequential(
            nn.BatchNorm2d(192),
            nn.Conv2d(192,384,kernel_size=2,stride=2,bias=False),)
        self.stem.append(res2)

        res3=nn.Sequential(
            nn.BatchNorm2d(384),
            nn.Conv2d(384,768,kernel_size=2,stride=2,bias=False),)
        self.stem.append(res3)
        
        self.stage_cfgs = [
            # expand_ratio, channels, # blocks, stride of first block
            [4,  96, 3, 1],
            [4,  192, 3, 1],
            [4,  384, 9, 1],
            [4,  768, 3, 1],
        ]

        # Remember that our stem left us off at 16 channels. We're going to 
        # keep updating this in_channels variable as we go
        in_channels = 96

        # Let's make the layers
        self.layers = nn.ModuleList()
        for curr_stage in self.stage_cfgs:
            expand_ratio, num_channels, num_blocks, stride = curr_stage
            layers = []
            for block_idx in range(num_blocks):
                in_channels=num_channels
                out_channels = num_channels
                layers.append(InvertedResidualBlock(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    # only have non-trivial stride if first block
                    stride=stride if block_idx == 0 else 1, 
                    expand_ratio=expand_ratio
                ))
                # In channels of the next block is the out_channels of the current one
                # in_channels = out_channels 
            layers = nn.Sequential(*layers)
            self.layers.append(layers) 
         # Done, save them to the class

        # Some final feature mixing
        self.final_block = nn.Sequential(
            nn.BatchNorm2d(768)
        )

        # Now, we need to build the final classification layer.
        self.cls_layer = nn.Sequential(
            # TODO: Fill this in!
            # Pool over & collapse the spatial dimensions to (1, 1)
            # Collapse the trivial (1, 1) dimensions
            # Project to our # of classes
            nn.AdaptiveAvgPool2d((1,1)),
            nn.BatchNorm2d(768),
            nn.Flatten(),
            nn.Linear(768,num_classes)
        )

        self._initialize_weights()

    def _initialize_weights(self):
        """
        Usually, I like to use default pytorch initialization for stuff, but
        MobileNetV2 made a point of putting in some custom ones, so let's just
        use them.
        """
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, nn.BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()

    def forward(self, x):
      
        out=x
        for i in range(4):
            # print(i)
            
            out=self.stem[i](out)
            # print(out.shape)
            out=self.layers[i](out)
        # out = self.final_block(out)
        out = self.cls_layer(out)
        return out

# Very Simple Network

# Dataset & DataLoader

In [ ]:
"""
Transforms (data augmentation) is quite important for this task.
Go explore https://pytorch.org/vision/stable/transforms.html for more details
"""
DATA_DIR = "/content"
TRAIN_DIR = osp.join(DATA_DIR, "classification/classification/train") # This is a smaller subset of the data. Should change this to classification/classification/train
VAL_DIR = osp.join(DATA_DIR, "classification/classification/dev")
TEST_DIR = osp.join(DATA_DIR, "classification/classification/test")

train_transforms = [ttf.ToTensor()]
val_transforms = [ttf.ToTensor()]

train_dataset = torchvision.datasets.ImageFolder(TRAIN_DIR,
                                                 transform=ttf.Compose(train_transforms))
val_dataset = torchvision.datasets.ImageFolder(VAL_DIR,
                                               transform=ttf.Compose(val_transforms))


train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          shuffle=True, drop_last=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False,
                        drop_last=True, num_workers=1)

# Setup everything for training

In [17]:
model = ConvNeXt()
#model.cuda()

# For this homework, we're limiting you to 35 million trainable parameters, as
# outputted by this. This is to help constrain your search space and maintain
# reasonable training times & expectations
num_trainable_parameters = 0
for p in model.parameters():
    num_trainable_parameters += p.numel()
print("Number of Params: {}".format(num_trainable_parameters))

# TODO: What criterion do we use for this task?
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=(len(train_loader) * epochs))
# T_max is "how many times will i call scheduler.step() until it reaches 0 lr?"

# For this homework, we strongly strongly recommend using FP16 to speed up training.
# It helps more for larger models.
# Go to https://effectivemachinelearning.com/PyTorch/8._Faster_training_with_mixed_precision
# and compare "Single precision training" section with "Mixed precision training" section
scaler = torch.cuda.amp.GradScaler()

Number of Params: 33158200


NameError: ignored

In [22]:
print(ConvNeXt())

ConvNeXt(
  (stem): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4), bias=False)
      (1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(96, 192, kernel_size=(2, 2), stride=(2, 2), bias=False)
    )
    (2): Sequential(
      (0): BatchNorm2d(192, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(192, 384, kernel_size=(2, 2), stride=(2, 2), bias=False)
    )
    (3): Sequential(
      (0): BatchNorm2d(384, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2), bias=False)
    )
  )
  (layers): ModuleList(
    (0): Sequential(
      (0): InvertedResidualBlock(
        (spatial_mixing): Sequential(
          (0): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 

In [ ]:
torch.save({'epoch': 20, 'model_state_dict': model.state_dict(),
            'criterion_state_dict': criterion.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
           'scheduler_state_dict': scheduler.state_dict(),
            'scaler_state_dict': scaler.state_dict()},
           f'/content/drive/MyDrive/IDL_Code_Model/HW2P2_Model/early_submission_model_epoch_{epoch}.pth', _use_new_zipfile_serialization = False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
WEIGHT_FILE = '/content/drive/MyDrive/IDL_Code_Model/HW2P2_Model/early_submission_model_epoch_19.pth'
checkpoint = torch.load(WEIGHT_FILE, map_location=device)
epoch = checkpoint['epoch']
model.load_state_dict(checkpoint['model_state_dict'])
criterion.load_state_dict(checkpoint['criterion_state_dict'])
scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
scaler.load_state_dict(checkpoint['scaler_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
print(model)

Network(
  (backbone): Sequential(
    (0): ZeroPad2d(padding=(1, 2, 1, 2), value=0.0)
    (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(4, 4))
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU(inplace=True)
    (4): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2))
    (6): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): ReLU(inplace=True)
    (8): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (9): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2))
    (10): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ZeroPad2d(padding=(1, 0, 1, 0), value=0.0)
    (13): Conv2d(256, 512, kernel_size=(3, 3), stride=(2, 2))
    (14): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (15): ReLU(inplace=True)
    (16): AvgPool2d(kernel_size=7, strid

# Let's train!

In [ ]:
for epoch in range(epochs):
    # Quality of life tip: leave=False and position=0 are needed to make tqdm usable in jupyter
    batch_bar = tqdm(total=len(train_loader), dynamic_ncols=True, leave=False, position=0, desc='Train') 

    num_correct = 0
    total_loss = 0

    for i, (x, y) in enumerate(train_loader):
        optimizer.zero_grad()

        x = x.cuda()
        y = y.cuda()

        # Don't be surprised - we just wrap these two lines to make it work for FP16
        with torch.cuda.amp.autocast():     
            outputs = model(x)
            loss = criterion(outputs, y)

        # Update # correct & loss as we go
        num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
        total_loss += float(loss)

        # tqdm lets you add some details so you can monitor training as you train.
        batch_bar.set_postfix(
            acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)),
            loss="{:.04f}".format(float(total_loss / (i + 1))),
            num_correct=num_correct,
            lr="{:.04f}".format(float(optimizer.param_groups[0]['lr'])))
        
        # Another couple things you need for FP16. 
        scaler.scale(loss).backward() # This is a replacement for loss.backward()
        scaler.step(optimizer) # This is a replacement for optimizer.step()
        scaler.update() # This is something added just for FP16

        scheduler.step() # We told scheduler T_max that we'd call step() (len(train_loader) * epochs) many times.

        batch_bar.update() # Update tqdm bar
    batch_bar.close() # You need this to close the tqdm bar

    # You can add validation per-epoch here if you would like

    print("Epoch {}/{}: Train Acc {:.04f}%, Train Loss {:.04f}, Learning Rate {:.04f}".format(
        epoch + 1,
        epochs,
        100 * num_correct / (len(train_loader) * batch_size),
        float(total_loss / len(train_loader)),
        float(optimizer.param_groups[0]['lr'])))

Epoch 1/20: Train Acc 0.0143%, Train Loss 8.8918, Learning Rate 0.0994


Epoch 2/20: Train Acc 0.0258%, Train Loss 8.7391, Learning Rate 0.0976


Epoch 3/20: Train Acc 0.0744%, Train Loss 8.5232, Learning Rate 0.0946


Epoch 4/20: Train Acc 0.1846%, Train Loss 8.2660, Learning Rate 0.0905


Epoch 5/20: Train Acc 0.3405%, Train Loss 7.9924, Learning Rate 0.0854


...

Epoch 16/20: Train Acc 22.6319%, Train Loss 4.7504, Learning Rate 0.0095


Epoch 17/20: Train Acc 26.1390%, Train Loss 4.5827, Learning Rate 0.0054


Epoch 18/20: Train Acc 28.9406%, Train Loss 4.4564, Learning Rate 0.0024


Epoch 19/20: Train Acc 30.9853%, Train Loss 4.3746, Learning Rate 0.0006


Epoch 20/20: Train Acc 32.1057%, Train Loss 4.3363, Learning Rate 0.0000


# Classification Task: Validation

In [ ]:
model.eval()
batch_bar = tqdm(total=len(val_loader), dynamic_ncols=True, position=0, leave=False, desc='Val')
num_correct = 0
for i, (x, y) in enumerate(val_loader):

    x = x.cuda()
    y = y.cuda()

    with torch.no_grad():
        outputs = model(x)

    num_correct += int((torch.argmax(outputs, axis=1) == y).sum())
    batch_bar.set_postfix(acc="{:.04f}%".format(100 * num_correct / ((i + 1) * batch_size)))

    batch_bar.update()
    
batch_bar.close()
print("Validation: {:.04f}%".format(100 * num_correct / len(val_dataset)))

Validation: 6.2571%


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Classification Task: Submit to Kaggle

In [ ]:
class ClassificationTestSet(Dataset):
    # It's possible to load test set data using ImageFolder without making a custom class.
    # See if you can think it through!

    def __init__(self, data_dir, transforms):
        self.data_dir = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in data_dir
        self.img_paths = list(map(lambda fname: osp.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        return self.transforms(Image.open(self.img_paths[idx]))

In [ ]:
test_dataset = ClassificationTestSet(TEST_DIR, ttf.Compose(val_transforms))
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False,
                         drop_last=False, num_workers=1)

In [ ]:
model.eval()
batch_bar = tqdm(total=len(test_loader), dynamic_ncols=True, position=0, leave=False, desc='Test')

res = []
for i, (x) in enumerate(test_loader):

    # TODO: Finish predicting on the test set.
    x = x.cuda()

    with torch.no_grad():
        outputs = model(x)

    res.extend(torch.argmax(outputs, axis=1))

    batch_bar.update()
    
batch_bar.close()

Validation: 6.2571%


In [ ]:
len(test_loader)

137

In [ ]:
len(res)

35000

In [ ]:
with open("classification_early_submission.csv", "w+") as f:
    f.write("id,label\n")
    for i in range(len(test_dataset)):
        f.write("{},{}\n".format(str(i).zfill(6) + ".jpg", res[i]))

In [ ]:
! kaggle competitions submit -c 11-785-s22-hw2p2-classification -f classification_early_submission.csv -m early

100% 541k/541k [00:00<00:00, 2.67MB/s]
Successfully submitted to Face Recognition

# Verification Task: Validation

There are 6K verification dev images, but 166K "pairs" for you to compare. So, it's much more efficient to compute the features for the 6K verification images, and just compare afterwards.

This will be done by creating a dictionary mapping the image file names to the features. Then, you'll use this dictionary to compute the similarities for each pair.

In [ ]:
!ls verification/verification/dev | wc -l
!cat verification/verification/verification_dev.csv | wc -l
!cat /content/verification_early_submission.csv | wc -l

6000
166801
667601


In [ ]:
class VerificationDataset(Dataset):
    def __init__(self, data_dir, transforms):
        self.data_dir = data_dir
        self.transforms = transforms

        # This one-liner basically generates a sorted list of full paths to each image in data_dir
        self.img_paths = list(map(lambda fname: osp.join(self.data_dir, fname), sorted(os.listdir(self.data_dir))))

    def __len__(self):
        return len(self.img_paths)
    
    def __getitem__(self, idx):
        # We return the image, as well as the path to that image (relative path)
        return self.transforms(Image.open(self.img_paths[idx])), osp.relpath(self.img_paths[idx], self.data_dir)

In [ ]:
val_veri_dataset = VerificationDataset(osp.join(DATA_DIR, "verification/verification/dev"),
                                       ttf.Compose(val_transforms))
val_ver_loader = torch.utils.data.DataLoader(val_veri_dataset, batch_size=batch_size, 
                                             shuffle=False, num_workers=1)

In [ ]:
model.eval()
# early_stop = 0

feats_dict = dict()
for batch_idx, (imgs, path_names) in tqdm(enumerate(val_ver_loader), total=len(val_ver_loader), position=0, leave=False):
    imgs = imgs.cuda()

    # print(len(path_names))
    # early_stop += 1

    with torch.no_grad():
        # Note that we return the feats here, not the final outputs
        # Feel free to try the final outputs too!
        feats = model(imgs, return_feats=True) 
    for emb_index in range(len(feats)):
        feats_dict[path_names[emb_index]] = feats[emb_index]
            
        # print(feats_dict)
    # if early_stop == 1:
      # break
    
    # TODO: Now we have features and the image path names. What to do with them?
    # Hint: use the feats_dict somehow.

In [ ]:
# What does this dict look like?
print(list(feats_dict.items())[0])
print(list(feats_dict.items())[0][1].shape)

('000b28b024.jpg', tensor([1.2569, 0.3534, 0.2190, 4.7730, 3.1352, 1.4041, 0.5100, 0.5319, 0.5085,
        0.9756, 2.9520, 0.6323, 0.6236, 0.5534, 0.2738, 0.5251, 0.3794, 0.5342,
        0.3272, 0.9717, 1.3553, 1.7167, 0.4047, 1.2829, 0.3706, 0.5238, 0.7662,
        0.4161, 1.2471, 1.3157, 3.1988, 0.2249, 0.4256, 4.0080, 0.4246, 2.1858,
        0.2173, 0.2536, 0.2647, 0.2485, 0.0968, 3.9496, 0.1020, 3.7258, 1.4275,
        0.0650, 0.5717, 1.3966, 0.1236, 0.2548, 0.8670, 0.4743, 0.9593, 2.1185,
        2.2810, 0.1006, 0.6388, 1.0772, 0.3143, 0.5435, 0.1444, 0.3195, 0.2811,
        0.6508, 1.0987, 1.8814, 2.0824, 0.2072, 0.3854, 0.6865, 0.2229, 1.5995,
        2.4710, 0.2762, 2.0633, 1.0991, 0.3682, 0.0956, 3.9837, 0.8518, 0.8782,
        2.4152, 1.0134, 0.7126, 0.1568, 0.5427, 0.8678, 4.2020, 0.9462, 0.1952,
        2.7003, 2.2646, 3.9707, 2.3978, 1.0811, 0.1821, 0.0913, 0.0999, 2.7244,
        0.8473, 0.4098, 0.1846, 0.1424, 0.1086, 0.2833, 0.7411, 0.1076, 0.4617,
        0.1577, 0.335

In [ ]:
input1 = torch.randn(2)
print(input1)
input2 = torch.randn(2)
print(input2)
cos = nn.CosineSimilarity(dim=0, eps=1e-6)
output = cos(input1, input1)
print(output.item())

tensor([ 0.2630, -0.3452])
tensor([-0.7791, -0.5361])
1.0


In [ ]:
# We use cosine similarity between feature embeddings.
# TODO: Find the relevant function in pytorch and read its documentation.
similarity_metric = nn.CosineSimilarity(dim=0, eps=1e-6)

val_veri_csv = osp.join(DATA_DIR, "verification/verification/verification_dev.csv")


# Now, loop through the csv and compare each pair, getting the similarity between them
pred_similarities = []
gt_similarities = []
for line in tqdm(open(val_veri_csv).read().splitlines()[1:], position=0, leave=False): # skip header
    img_path1, img_path2, gt = line.split(",")

    # TODO: Use the similarity metric
    # How to use these img_paths? What to do with the features?
    # similarity = similarity_metric(...)
    pred_similarities.append(similarity_metric(feats_dict[img_path1[4:]], feats_dict[img_path2[4:]]).item())
    gt_similarities.append(int(gt))

pred_similarities = np.array(pred_similarities)
gt_similarities = np.array(gt_similarities)

print("AUC:", roc_auc_score(gt_similarities, pred_similarities))

AUC: 0.7949669511548569


# Verification Task: Submit to Kaggle

In [ ]:
test_veri_dataset = VerificationDataset(osp.join(DATA_DIR, "verification/verification/test"),
                                        ttf.Compose(val_transforms))
test_ver_loader = torch.utils.data.DataLoader(test_veri_dataset, batch_size=batch_size, 
                                              shuffle=False, num_workers=1)

In [ ]:
model.eval()

feats_dict = dict()
for batch_idx, (imgs, path_names) in tqdm(enumerate(test_ver_loader), total=len(test_ver_loader), position=0, leave=False):
    imgs = imgs.cuda()

    with torch.no_grad():
        # Note that we return the feats here, not the final outputs
        # Feel free to try to final outputs too!
        feats = model(imgs, return_feats=True) 
    
    # TODO: Now we have features and the image path names. What to do with them?
    # Hint: use the feats_dict somehow.
    for emb_index in range(len(feats)):
        feats_dict[path_names[emb_index]] = feats[emb_index]



In [ ]:
# We use cosine similarity between feature embeddings.
# TODO: Find the relevant function in pytorch and read its documentation.
similarity_metric = nn.CosineSimilarity(dim=0, eps=1e-6)

val_veri_csv = osp.join(DATA_DIR, "verification/verification/verification_test.csv")


# Now, loop through the csv and compare each pair, getting the similarity between them
pred_similarities = []
for line in tqdm(open(val_veri_csv).read().splitlines()[1:], position=0, leave=False): # skip header
    img_path1, img_path2 = line.split(",")

    pred_similarities.append(similarity_metric(feats_dict[img_path1[5:]], feats_dict[img_path2[5:]]).item())
    # TODO: Finish up verification testing.
    # How to use these img_paths? What to do with the features?

In [ ]:
with open("verification_early_submission.csv", "w+") as f:
    f.write("id,match\n")
    for i in range(len(pred_similarities)):
        f.write("{},{}\n".format(i, pred_similarities[i]))

In [ ]:
!kaggle competitions submit -c 11-785-s22-hw2p2-verification -f verification_early_submission.csv -m early_submission

100% 16.4M/16.4M [00:03<00:00, 4.53MB/s]
Successfully submitted to Face Verification

In [ ]:
!kaggle competitions submit -c 11-785-s22-hw2p2-verification -f verification_sample_submission.csv -m early

100% 6.90M/6.90M [00:03<00:00, 1.82MB/s]
Successfully submitted to Face Verification

# Extras

In [ ]:
# If you keep re-initializing your model in Colab, can run out of GPU memory, need to restart.
# These three lines can help that - run this before you re-initialize your model

del model
torch.cuda.empty_cache()
!nvidia-smi

Thu Feb 24 10:34:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    65W / 149W |   1012MiB / 11441MiB |     89%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------